In [1]:
import pandas as pd
df=pd.read_csv('quora_duplicate_questions.tsv',sep='\t')
df.dropna(inplace=True)
df = df.iloc[100000:150000,:]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 100000 to 150000
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            50000 non-null  int64 
 1   qid1          50000 non-null  int64 
 2   qid2          50000 non-null  int64 
 3   question1     50000 non-null  object
 4   question2     50000 non-null  object
 5   is_duplicate  50000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 2.7+ MB


In [2]:
df.drop(columns=['id','qid1','qid2','is_duplicate'],inplace=True)

In [3]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

def predict_emotion(text):
    if(model(text)[0]['score']<0.5):
        return('neutral')
    else:
        return model(text)[0]['label']


In [5]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['question1']
    target_text = row['question2']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    if(input_emotion==target_emotion or input_emotion=='neutral' or target_emotion=='neutral'):
        df.drop(index, inplace=True)

    else:
        if(total_grad.index(input_emotion)>total_grad.index(target_emotion)):
            df.at[index,'question1'],df.at[index,'question2']=df.at[index,'question2'],df.at[index,'question1']
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'question1'] = modified_input_text

Processing rows: 100%|██████████| 50000/50000 [1:13:42<00:00, 11.31it/s]


In [6]:
df.to_csv('quora3.csv',index=False)

In [8]:
df

,question1,question2
100012,curiosity to desire: Was Cleopatra highly imag...,Was Cleopatra highly image-conscious and willi...
100022,love to curiosity: Can parents love their adop...,Can you ever love your adopted child as much a...
100037,curiosity to love: What is the difference betw...,What is the difference between depression and ...
100066,curiosity to confusion: How do you build a com...,How do you build a company that profits from d...
100088,curiosity to embarrassment: What are the most ...,What's the most awkward/embarrassing situation...
...,...,...
149933,curiosity to admiration: How can I increase my...,How can I increase my presence of mind?
149953,anger to curiosity: Why doesn't India dare to ...,Why doesn't India dare to invade Pakistan?
149958,confusion to curiosity: Doubt on hair removal....,What is the history of hair removal?
149963,curiosity to confusion: What happens to your F...,What happens to your FICO score when you add a...
